In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from PIL import Image
from scipy.signal import savgol_filter
        
#importing_data
n_fs = pd.read_csv('../input/glass-data/Fused_silica.csv',header=1)
n_fs = n_fs.values;

n_sl = pd.read_csv('../input/glass-data/Soda_lime.csv',header=1)
n_sl = n_sl.values;

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

plt.plot(n_fs[:,0],n_fs[:,1],n_sl[:,0],n_sl[:,1])
plt.legend(['Fused Silica','Soda Lime'])
plt.title('Refractive Index')
plt.xlabel('Wavelength [um]')
plt.ylabel('n')

# wavelength in [nm]
lam_start = 350
lam_stop = 2500
nb_pts_lam = 300

# angles in [deg]
theta_start = 0
theta_stop = 70
nb_pts_ang = 1
theta0 = theta_start

lam_t = np.linspace(lam_start*1e-9,lam_stop*1e-9,nb_pts_lam)
theta_t = np.linspace(theta_start*np.pi/180,theta_stop*np.pi/180,nb_pts_ang)

#thickness of nanostructures in [nm]
thick_etch1 = 0e-9
d_glass = 400e-9
thick_etch2 = 0e-9

#refractive index
n_air = 1
n_glass = np.interp(lam_t, n_fs[:,0]*1e-6, n_fs[:,1])

#physics constants
e0 = 8.854e-12
u0 = 1.256e-6

mfp = 10e-9;

def layer_in_p(n1,n2,n3,d,k,theta):
    theta1 = np.conj(np.arcsin(n_air*np.sin(theta)/n1))
    theta2 = np.conj(np.arcsin(n_air*np.sin(theta)/n2))
    theta3 = np.conj(np.arcsin(n_air*np.sin(theta)/n3))
    phi1 = k*n2*d*np.cos(theta)
    r_p = (n2*np.cos(theta1)-n1*np.cos(theta2))/(n2*np.cos(theta1)+n1*np.cos(theta2))
    t_p = (2*n2*np.cos(theta2))/(n3*np.cos(theta2)+n2*np.cos(theta3))
    M1 = np.array([[1,np.abs(r_p)**2],[np.abs(r_p)**2,1]])
    P = np.array([[np.abs(np.exp(1j*phi1))**2,0],[0,np.abs(np.exp(-1j*phi1))**2]])
    M2 = np.array([[np.abs(1/t_p)**2,0],[0,np.abs(1/t_p)**2]])
    Mat = np.matmul(M1,P)
    return np.matmul(Mat,M2)

def layer_in_s(n1,n2,n3,d,k,theta):
    theta1 = np.conj(np.arcsin(n_air*np.sin(theta)/n1))
    theta2 = np.conj(np.arcsin(n_air*np.sin(theta)/n2))
    theta3 = np.conj(np.arcsin(n_air*np.sin(theta)/n3))
    phi1 = k*n2*d*np.cos(theta)
    r_s = (n1*np.cos(theta1)-n2*np.cos(theta2))/(n1*np.cos(theta1)+n2*np.cos(theta2))
    t_s = (2*n2*np.cos(theta2))/(n2*np.cos(theta2)+n3*np.cos(theta3))
    M1 = np.array([[1,np.abs(r_s)**2],[np.abs(r_s)**2,1]])
    P = np.array([[np.abs(np.exp(1j*phi1))**2,0],[0,np.abs(np.exp(-1j*phi1))**2]])
    M2 = np.array([[np.abs(1/t_s)**2,0],[0,np.abs(1/t_s)**2]])
    Mat = np.matmul(M1,P)
    return np.matmul(Mat,M2)

def t0_in_p(n1,n2,theta):
    return (2*n1*np.cos(theta))/(n2*np.cos(theta)+n1*np.sqrt(1-(n1*np.sin(theta)/n2)**2))
def rf_in_p(n1,n2,theta):
    return (n2*np.cos(theta)-n1*np.sqrt(1-(n1*np.sin(theta)/n2)**2))/(n1*np.sqrt(1-(n1*np.sin(theta)/n2)**2)+n2*np.cos(theta))

def t0_in_s(n1,n2,theta):
    return (2*n1*np.cos(theta))/(n1*np.cos(theta)+n2*np.sqrt(1-(n1*np.sin(theta)/n2)**2))
def rf_in_s(n1,n2,theta):
    return (n1*np.cos(theta)-n2*np.sqrt(1-(n1*np.sin(theta)/n2)**2))/(n1*np.cos(theta)+n2*np.sqrt(1-(n1*np.sin(theta)/n2)**2))

In [18]:
r_123o1 = pd.read_csv('../input/refl-123o1/123-O1.txt', delimiter = "\t",header=None)
im = Image.open('../input/123o1-image/123-O1_0011.tif')


In [19]:
def cal_in_p(index,ds,lam,theta):
    k = 2*np.pi/lam
    t0 = t0_in_p(index[0],index[1],theta)
    M = 1/np.abs(t0)**2*np.array([[1,0],[0,1]])
    for l in range(len(ds)):
        layer_p = layer_in_p(index[l],index[l+1],index[l+2],ds[l],k,theta)
        M = np.matmul(M,layer_p)
    rf = rf_in_p(index[-2],index[-1],theta)  
    Mt = np.array([[1,np.abs(rf)**2],[np.abs(rf)**2,1]]);
    M = np.matmul(M,Mt)
    return M[1,0]/M[0,0] 
    
def cal_in_s(index,ds,lam,theta):
    k = 2*np.pi/lam
    t0 = t0_in_s(index[0],index[1],theta)
    M = 1/np.abs(t0)**2*np.array([[1,0],[0,1]])
    for l in range(len(ds)):
        layer_p = layer_in_s(index[l],index[l+1],index[l+2],ds[l],k,theta)
        M = np.matmul(M,layer_p)
    rf = rf_in_s(index[-2],index[-1],theta)  
    Mt = np.array([[1,np.abs(rf)**2],[np.abs(rf)**2,1]]);
    M = np.matmul(M,Mt)
    return M[1,0]/M[0,0] 
    
    
def cal_t(glass_type,thick1,d_glass,thick2):  
    r_tp = np.array([])
    for theta in theta_t:
        r_lam = np.array([])
        for lam in lam_t:
            n_glass = np.interp(lam, glass_type[:,0]*1e-6, glass_type[:,1])
            index_profile,d_profile = index_prof(n_air,n_glass,thick1,d_glass,thick2,lam)
            r_p1 = cal_in_p(index_profile,d_profile,lam,theta)
            r_s1 = cal_in_s(index_profile,d_profile,lam,theta)
            r_lam = np.append(r_lam,((r_p1+r_s1)/2))
    if theta == theta_t[0]:
        r_t = r_lam
    else:
        r_t = np.vstack([r_t,r_lam])
                              
    return r_t
 
def index_prof(n1,n2,thick1,d_glass,thick2,lam):
    index_profile = np.array([])
    d_profile = np.array([])
    index_profile = np.append(index_profile,n_air)
    mfp = lam/factor
    nb_layers = int(np.floor(thick1/mfp))
    if nb_layers >0:
        d = thick1/nb_layers
        index_func = pyr_index(ang,r2,r3,r4,n1,n2,plot=False)
        index_layers = index_step(index_func,nb_layers)
        d_profile = np.append(d_profile,np.linspace(d,d,nb_layers))
        index_profile = np.append(index_profile,index_layers)
    if d_glass > 0:
        index_profile = np.append(index_profile,n2)
        d_profile = np.append(d_profile,d_glass)
        
    nb_layers = int(np.floor(thick2/mfp))
    if nb_layers >0:
        d = thick2/nb_layers
        index_func = pyr_index(ang,r2,r3,r4,n2,n1,plot=False)
        index_layers = index_step(index_func,nb_layers)
        d_profile = np.append(d_profile,np.linspace(d,d,nb_layers))
        index_profile = np.append(index_profile,index_layers)
    index_profile = np.append(index_profile,n1)
    return index_profile,d_profile  

def pyr_index(ang,r2,r3,r4,n1,n2,plot=True):
    ang = ang*np.pi/180
    sl = -1/np.tan(ang)
    b = (1-r3)/(np.tan(ang))
    x1 = np.arange(0,1,0.01)
    y1 = np.zeros(len(x1))

    for xx in x1:
        if np.abs(xx)<r2:
            y1[np.where(x1 == xx)] = sl*r2+b
        elif np.abs(xx)>(1-r4):
            y1[np.where(x1 == xx)] = 0
        else:
            y1[np.where(x1 == xx)] = sl*xx+b
    
    if plot:
        x1 = np.arange(0,1,0.01);x3 = np.arange(2,3,0.01);x5 = np.arange(4,5,0.01)
        x2 = np.arange(-1,0,0.01);x4 = np.arange(1,2,0.01);x6 = np.arange(3,4,0.01)    
        y2 = y1[-1::-1]
        y = np.concatenate([y2,y1,y2,y1,y2,y1]);x = np.concatenate([x2,x1,x4,x3,x6,x5])
        plt.plot(x,y);
        plt.gca().set_aspect('equal')
        
    #area calculation
    z1 = sl*r2+b
    zz = np.arange(z1,0,-0.01);
    xx = np.linspace(0,1,len(zz))
    area = ((zz-b)*2/sl)**2
    index = n2*area/4+n1*(4-area)/4
    index[0] = n1
    return index

def index_step(index,nb_layers):
    xx = np.linspace(0,thickness,len(index))
    xx2 = np.linspace(0,thickness,nb_layers)
    index_profile = np.interp(xx2,xx,index)
    return index_profile


In [20]:
ang = 10;
r2 = 0.05             #  flat top pyramid
r3 = 0.0                #  distance between pyramids
r4 = r3+0.0;              #  flat side 
x1 = np.arange(0,1,0.01);x3 = np.arange(2,3,0.01);x5 = np.arange(4,5,0.01)
x2 = np.arange(-1,0,0.01);x4 = np.arange(1,2,0.01);x6 = np.arange(3,4,0.01)
y1 = np.zeros(len(x1))
thickness = 200e-9;

sl = -1/np.tan(ang)
b = (1-r3)/(np.tan(ang))

for xx in x1:
    if np.abs(xx)<r2:
        y1[np.where(x1 == xx)] = sl*r2+b
    elif np.abs(xx)>(1-r4):
        y1[np.where(x1 == xx)] = 0
    else:
        y1[np.where(x1 == xx)] = sl*xx+b


In [21]:
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.subplot(1,2,1)

ang = 5; r2=0.5;r3=0.1;r4=r3;
index1 = pyr_index(ang,r2,r3,r4,1,1.51,plot=True)
nb_layers = int(thickness/mfp)
index_profile1= index_step(index1,nb_layers)

imarray=np.array(im)
plt.subplot(1,2,2)
plt.imshow(imarray[0:400,0:1200])

In [22]:
plt.plot(index_profile1)

In [23]:
thick1 = 200e-9;d_glass =1e-3;thick2 = 0e-9;
mfp = 20e-9;factor = 20
r_t1 = cal_t(n_fs,thick1,d_glass,thick2)
window = 101
order = 1
r_t1b = savgol_filter(r_t1, window, order)
plt.plot(lam_t,r_t1b)

plt.plot(r_123o1[0]*1e-9,(100-r_123o1[1])/100)